In [1]:
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalMaxPool1D, merge
from keras.layers import LSTM, Bidirectional, MaxPooling1D, Embedding, Dropout, Flatten, CuDNNLSTM, Conv1D, BatchNormalization
from keras.models import Model, load_model
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from keras.optimizers import Adam
# from google.colab import drive
from keras.callbacks import ModelCheckpoint
!pip install keras-self-attention

import keras
from keras_self_attention import SeqSelfAttention
# drive.mount('/content/gdrive')

Using TensorFlow backend.


  Stored in directory: C:\Users\noden\AppData\Local\pip\Cache\wheels\7b\05\a0\99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention


In [33]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip
!ls
!pwd

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.
'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
MAX_SEQUENCE_LENGTH = 1200
MAX_VOCAB_SIZE = 100000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 128
EPOCHS = 80

In [5]:
import json
import requests
import os
from pathlib import Path

data_dir = "project-files"

def get_tweet_text_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data

def get_events(event_dir):
    event_list = []
    text_list = []
    label_list = []

    data = get_tweet_text_from_json("train.json")
    for k, v in data.items():
        text_list.append(v["text"])
        label_list.append(v["label"])
        
    data1 = get_tweet_text_from_json("train_data.json")
    for i in range(len(data1)):
        if i < 1092:
            text_list.append(data1[i])
            label_list.append(1)
        else:
            text_list.append(data1[i])
            label_list.append(0)          

    return text_list, label_list

test = []
test_data = get_tweet_text_from_json("test-unlabelled.json")
for k, v in test_data.items():
    test.append(v["text"])

dev = []
dev_targets = []
data2 = get_tweet_text_from_json("dev.json")
for k, v in data2.items():
    dev.append(v["text"])
    dev_targets.append(v["label"])

text_list, label_list = get_events(data_dir)
print(len(text_list))

3974


In [6]:
filter_ = []
for i in range(len(test)):
    if 'climate' in test[i] or 'Climate Change' in test[i] or 'Earth Hour' in test[i] or 'Energy Production' in test[i] \
    or "Global Warming" in test[i] or "CO2" in test[i] or "greenhouse gas emission" in test[i] or "fossil fuel" in test[i] \
    or "ipcc" in test[i] or "indc" in test[i] or "unfccc" in test[i] or "sea-level" in test[i]:
        if 'economic climate' not in test[i]:
            filter_.append(1)
        else:
            filter_.append(0)
    else:
        filter_.append(0)

filter_2 = []
for i in range(len(dev)):
    if 'climate' in dev[i] or 'Climate Change' in dev[i] or 'Earth Hour' in dev[i] or 'Energy Production' in dev[i] \
    or "Global Warming" in dev[i] or "CO2" in dev[i] or "greenhouse gas emission" in dev[i] or "fossil fuel" in dev[i] \
    or "ipcc" in dev[i] or "indc" in dev[i] or "unfccc" in dev[i] or "sea-level" in dev[i]:
        if 'economic climate' not in dev[i]:
            filter_2.append(1)
        else:
            filter_2.append(0)
    else:
        filter_2.append(0)
print(len(filter_2))

100


In [9]:
print('Loading word vectors...')
word2vec = {}
with open("glove.42B.300d.txt", encoding="utf8") as f:
    # is just a space-separated text file in the format:
    # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 1917494 word vectors.


In [10]:
sentences, targets = text_list, label_list
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
sequences, targets = shuffle(sequences, targets)
print(sequences[0])

word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))

# pad sequences so that we get a N x T matrix
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#print('Shape of data tensor:', data.shape)

# prepare embedding matrix
j = 0
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros.
            embedding_matrix[i] = embedding_vector
            j += 1

print("number of words covered by the dictionary: ", j)
print("total number of distinct words: ", len(word2idx))

[1, 1666, 956, 1470, 7, 6, 13340, 2, 9, 1861, 3, 307, 4, 1, 2046, 27, 133, 3, 979, 92, 54, 822, 1692, 16, 943, 223, 8478, 17, 9, 20, 1, 269, 4, 1855, 893, 2643, 560, 3, 18, 12482, 1, 2517, 4, 2327, 787, 265, 89, 15, 19, 13206, 2675, 5624, 87, 435, 86, 133, 3, 18, 1115, 35, 79, 3, 234, 1, 4643, 430, 1692, 15, 27, 4251, 16, 19, 225, 1, 467, 306, 1107, 727, 11889, 27, 14, 5, 247, 6, 366, 4, 184, 225, 5, 2161, 17, 1, 391, 27, 540, 1610, 6, 561, 3, 1, 2674, 893, 11, 1692, 223, 18, 11, 21385, 184, 322, 4, 570, 4, 4281, 19, 1310, 982, 19, 26, 1, 9705, 3605, 8, 12097, 1, 309, 371, 3, 1, 32, 672, 225, 50, 26, 7267, 1, 467, 306, 3058, 6278, 22, 8, 82, 4, 1485, 125, 1124, 88, 79, 97, 15503, 8828, 4, 969, 26, 1011, 10, 1656, 19, 7, 97, 697, 4, 57, 30, 639, 57, 97, 29, 24858, 17, 9, 20, 30, 234, 6, 4643, 430, 1692, 1, 586, 2, 9, 20, 8, 7, 1261, 32, 196, 240, 13, 24, 23, 1410, 463, 863, 1, 1666, 956, 1470, 7, 6, 13340, 2, 9, 1861, 3, 307, 4, 1, 2046, 27, 133, 3, 979, 92, 54, 822, 45, 219, 366, 45, 3

In [11]:
#data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
dev_sequences = tokenizer.texts_to_sequences(dev)
#For test set, comment out the above section and uncomment the line below
test_sequences = tokenizer.texts_to_sequences(test)
targets = targets + dev_targets
new_sequences = sequences + dev_sequences
new_data = pad_sequences(new_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(len(sequences), len(new_sequences))

3974 4074


In [27]:
print('Building model...')
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=True
)
rate_drop_lstm = 0.25
rate_drop_dense = 0.25
num_dense = 256
# train a 1D convnet with global maxpooling
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 4, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Bidirectional(LSTM(32, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm, return_sequences=True))(x)
merged = SeqSelfAttention(
    attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
    attention_activation=None,
    kernel_regularizer=keras.regularizers.l2(1e-6),
    use_attention_bias=False,
    name='Attention')(x)
merged = Flatten()(merged)
x = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=input_, outputs=x)

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


Building model...


In [ ]:
# keep repeating this cell and the above cell until a desired result is reached
r = model.fit(
  np.array(new_data),
  targets,
  batch_size=128,
  epochs=4,
  validation_split=100/4074,
  #callbacks=callbacks_list
)

p = model.predict(pad_sequences(dev_sequences, maxlen=MAX_SEQUENCE_LENGTH))

test_dict = {}
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(p)):
    text = "test-" + str(i)
    if (p[i][0] * filter_2[i]) > 0.5:
        if (dev_targets[i] ==  1):
            true_positive += 1
        if (dev_targets[i] ==  0):
            false_positive +=1
        test_dict[text] = {"label": 1}
    else:
        if (dev_targets[i] ==  1):
            false_negative += 1
        test_dict[text] = {"label": 0}
precision = true_positive/(true_positive+false_positive)
recall = true_positive/(true_positive+false_negative)
print(true_positive, false_positive, false_negative)
print("F1-score = ", 2 * precision * recall / (precision + recall))
print("precision = ", precision)
print("recall = ", recall)

In [32]:
# Test
p2 = model.predict(pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH))

test_dict = {}
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(p)):
    text = "test-" + str(i)
    if (p[i][0] * filter_[i]) > 0.5:
        test_dict[text] = {"label": 1}
    else:
        test_dict[text] = {"label": 0}

print(test_dict)

{'test-0': {'label': 0}, 'test-1': {'label': 0}, 'test-2': {'label': 0}, 'test-3': {'label': 0}, 'test-4': {'label': 0}, 'test-5': {'label': 0}, 'test-6': {'label': 0}, 'test-7': {'label': 0}, 'test-8': {'label': 0}, 'test-9': {'label': 0}, 'test-10': {'label': 0}, 'test-11': {'label': 0}, 'test-12': {'label': 0}, 'test-13': {'label': 0}, 'test-14': {'label': 0}, 'test-15': {'label': 0}, 'test-16': {'label': 0}, 'test-17': {'label': 0}, 'test-18': {'label': 0}, 'test-19': {'label': 0}, 'test-20': {'label': 0}, 'test-21': {'label': 0}, 'test-22': {'label': 0}, 'test-23': {'label': 0}, 'test-24': {'label': 0}, 'test-25': {'label': 0}, 'test-26': {'label': 0}, 'test-27': {'label': 0}, 'test-28': {'label': 0}, 'test-29': {'label': 0}, 'test-30': {'label': 0}, 'test-31': {'label': 0}, 'test-32': {'label': 0}, 'test-33': {'label': 0}, 'test-34': {'label': 0}, 'test-35': {'label': 0}, 'test-36': {'label': 0}, 'test-37': {'label': 0}, 'test-38': {'label': 0}, 'test-39': {'label': 0}, 'test-40'

In [0]:
import pickle
with open('./gdrive/My Drive/Colab/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model2 = load_model("./gdrive/My Drive/Colab/final-05-0.87.h5", custom_objects={'SeqSelfAttention':SeqSelfAttention})
model2.compile(
  loss='binary_crossentropy',
  optimizer=Adam(),
  metrics=['accuracy']
)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
import json

with open('test-output.json', 'w') as fp:
    json.dump(test_dict, fp)

In [0]:
from google.colab import files

files.download('test-output.json')